In [2]:
import os
import os.path as osp
import json
import math
from tqdm import tqdm
from glob import glob
from pprint import pprint
import copy

import torch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import cv2
import albumentations as A
import lanms
from albumentations.pytorch import ToTensorV2
from imageio import imread

from model import EAST
from detect import detect

### polygon -> rectangle(외접하는 직사각형, 회전X)

In [9]:
add_data_dir = os.environ.get("SM_CHANNEL_TRAIN", "../input/data/camper")

with open(osp.join(add_data_dir, "ufo/annotation.json"), "r") as f:
    anno = json.load(f)

anno = anno["images"]
anno_temp = copy.deepcopy(anno)

for img_name, img_info in tqdm(anno.items()):
    # annotation이 없는 이미지 삭제
    if img_info["words"] == {}:
        del anno_temp[img_name]
        continue
    for obj, obj_info in img_info["words"].items():
        if len(img_info["words"][obj]["points"]) == 4:
            continue
        # 점이 홀수개인 텍스트박스 삭제
        elif len(img_info["words"][obj]["points"]) % 2 == 1:
            del anno_temp[img_name]["words"][obj]

            if anno_temp[img_name]["words"] == {}:
                del anno_temp[img_name]
                continue

        # polygon을 x, y의 min, max로 외접하는 직사각형으로 변환
        else:
            x_points = [
                anno_temp[img_name]["words"][obj]["points"][i][0]
                for i in range(len(anno_temp[img_name]["words"][obj]["points"]))
            ]
            y_points = [
                anno_temp[img_name]["words"][obj]["points"][i][1]
                for i in range(len(anno_temp[img_name]["words"][obj]["points"]))
            ]
            x_min, x_max = min(x_points), max(x_points)
            y_min, y_max = min(y_points), max(y_points)
            anno_temp[img_name]["words"][obj]["points"] = [
                [x_min, y_min],
                [x_max, y_min],
                [x_max, y_max],
                [x_min, y_max]
            ]

            if anno_temp[img_name]["words"] == {}:
                del anno_temp[img_name]
                continue

anno = {"images": anno_temp}

ufo_dir = osp.join("../input/data/camper", "ufo")
with open(osp.join(ufo_dir, "train.json"), "w") as f:
    json.dump(anno, f, indent=4)

100%|██████████| 1510/1510 [00:00<00:00, 52540.14it/s]


### polygon 삭제

In [9]:
add_data_dir = os.environ.get("SM_CHANNEL_TRAIN", "../input/data/camper")

with open(osp.join(add_data_dir, "ufo/annotation.json"), "r") as f:
    anno = json.load(f)

anno = anno["images"]
anno_temp = copy.deepcopy(anno)

for img_name, img_info in tqdm(anno.items()):
    # annotation이 없는 이미지 삭제
    if img_info["words"] == {}:
        del anno_temp[img_name]
        continue
    for obj, obj_info in img_info["words"].items():
        if len(img_info["words"][obj]["points"]) == 4:
            continue
        # 점이 홀수개인 텍스트박스 삭제
        elif len(img_info["words"][obj]["points"]) % 2 == 1:
            del anno_temp[img_name]["words"][obj]

            if anno_temp[img_name]["words"] == {}:
                del anno_temp[img_name]
                continue

        # polygon을 삭제
        else:
            del anno_temp[img_name]["words"][obj]
            if anno_temp[img_name]["words"] == {}:
                del anno_temp[img_name]
                continue

anno = {"images": anno_temp}

ufo_dir = osp.join("../input/data/camper", "ufo")
with open(osp.join(ufo_dir, "train.json"), "w") as f:
    json.dump(anno, f, indent=4)

100%|██████████| 1510/1510 [00:00<00:00, 79271.53it/s]


### 양 끝 4개의 점으로 사각형 만들기
* point들이 annotation 가이드에 맞게 찍혀있다고 가정
* 모두 가로쓰기로 가정하고 사각형 위, 아래에 동일한 개수의 점이 있다고 가정

In [1]:
add_data_dir = os.environ.get("SM_CHANNEL_TRAIN", "../input/data/camper")

with open(osp.join(add_data_dir, "ufo/annotation.json"), "r") as f:
    anno = json.load(f)

anno = anno["images"]
anno_temp = copy.deepcopy(anno)

for img_name, img_info in tqdm(anno.items()):
    # annotation이 없는 이미지 삭제
    if img_info["words"] == {}:
        del anno_temp[img_name]
        continue
    for obj, obj_info in img_info["words"].items():
        if len(img_info["words"][obj]["points"]) == 4:
            continue
        # 점이 홀수개인 텍스트박스 삭제
        elif len(img_info["words"][obj]["points"]) % 2 == 1:
            del anno_temp[img_name]["words"][obj]

            if anno_temp[img_name]["words"] == {}:
                del anno_temp[img_name]
                continue

        # polygon의 양 끝 4점을 추출하여 사각형 생성
        else:
            anno_temp[img_name]["words"][obj]["points"] = [
                anno_temp[img_name]["words"][obj]["points"][0],
                anno_temp[img_name]["words"][obj]["points"][
                    (len(anno_temp[img_name]["words"][obj]["points"]) // 2 - 1)
                ],
                anno_temp[img_name]["words"][obj]["points"][
                    (len(anno_temp[img_name]["words"][obj]["points"]) // 2)
                ],
                anno_temp[img_name]["words"][obj]["points"][-1],
            ]

            if anno_temp[img_name]["words"] == {}:
                del anno_temp[img_name]
                continue

anno = {"images": anno_temp}

ufo_dir = osp.join("../input/data/camper", "ufo")
with open(osp.join(ufo_dir, "train.json"), "w") as f:
    json.dump(anno, f, indent=4)

100%|██████████| 1510/1510 [00:00<00:00, 82433.93it/s]


### 양 끝 4개의 점으로 사각형 만들기, 세로쓰기 구분

In [ ]:
add_data_dir = os.environ.get("SM_CHANNEL_TRAIN", "../input/data/camper")

with open(osp.join(add_data_dir, "ufo/annotation.json"), "r") as f:
    anno = json.load(f)

anno = anno["images"]
anno_temp = copy.deepcopy(anno)

for img_name, img_info in tqdm(anno.items()):
    # annotation이 없는 이미지 삭제
    if img_info["words"] == {}:
        del anno_temp[img_name]
        continue
    for obj, obj_info in img_info["words"].items():
        if len(img_info["words"][obj]["points"]) == 4:
            continue
        # 점이 홀수개인 텍스트박스 삭제
        elif len(img_info["words"][obj]["points"]) % 2 == 1:
            del anno_temp[img_name]["words"][obj]

            if anno_temp[img_name]["words"] == {}:
                del anno_temp[img_name]
                continue

        # polygon의 양 끝 4점을 추출하여 사각형 생성
        else:
            if anno_temp[img_name]["words"][obj]["orientation"] == "Vertical":
                anno_temp[img_name]["words"][obj]["points"] = [
                    anno_temp[img_name]["words"][obj]["points"][0],
                    anno_temp[img_name]["words"][obj]["points"][1],
                    anno_temp[img_name]["words"][obj]["points"][
                        (len(anno_temp[img_name]["words"][obj]["points"]) // 2)
                    ],
                    anno_temp[img_name]["words"][obj]["points"][
                        (len(anno_temp[img_name]["words"][obj]["points"]) // 2 + 1)
                    ],
                ]
            else:
                anno_temp[img_name]["words"][obj]["points"] = [
                    anno_temp[img_name]["words"][obj]["points"][0],
                    anno_temp[img_name]["words"][obj]["points"][
                        (len(anno_temp[img_name]["words"][obj]["points"]) // 2 - 1)
                    ],
                    anno_temp[img_name]["words"][obj]["points"][
                        (len(anno_temp[img_name]["words"][obj]["points"]) // 2)
                    ],
                    anno_temp[img_name]["words"][obj]["points"][-1],
                ]

            if anno_temp[img_name]["words"] == {}:
                del anno_temp[img_name]
                continue

anno = {"images": anno_temp}

ufo_dir = osp.join("../input/data/camper", "ufo")
with open(osp.join(ufo_dir, "train.json"), "w") as f:
    json.dump(anno, f, indent=4)

### 양 끝 4개의 점으로 사각형 -> 사각형 외부에 점이 존재하는 경우 사각형 확대
* 각 점과 사각형의 거리를 계산하여 사각형에서 가장 많이 벗어난 점을 찾아 변을 평행이동하여 사각형 확대

In [3]:
def cal_dist(x1, y1, x2, y2, x0, y0):
    """각 point가 새로 생성된 사각형 외부에 위치하는지 확인하기 위한 거리 계산"""
    area = abs((x1 - x0) * (y2 - y0) - (y1 - y0) * (x2 - x0))
    AB = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
    distance = area / AB
    return distance


def get_crosspt(x0, y0, x11, y11, x12, y12, x21, y21, x22, y22):
    """사각형의 변을 외부에 위치한 점을 지나게 평행이동하여 사각형의 꼭짓점을 반환"""
    if x12 == x11 or x22 == x21:
        if x12 == x11:
            cx = x0
            m2 = (y22 - y21) / (x22 - x21)
            cy = m2 * (cx - x21) + y21
            return [cx, cy]
        if x22 == x21:
            cx = x22
            m1 = (y12 - y11) / (x12 - x11)
            cy = m1 * (cx - x0) + y0
            return [cx, cy]
    m1 = (y12 - y11) / (x12 - x11)
    m2 = (y22 - y21) / (x22 - x21)
    if m1 == m2:
        return None
    cx = (x0 * m1 - y0 - x21 * m2 + y21) / (m1 - m2)
    cy = m1 * (cx - x0) + y0
    return [cx, cy]


add_data_dir = os.environ.get("SM_CHANNEL_TRAIN", "../input/data/camper")

with open(osp.join(add_data_dir, "ufo/annotation.json"), "r") as f:
    anno = json.load(f)

anno = anno["images"]
anno_temp = copy.deepcopy(anno)

for img_name, img_info in tqdm(anno.items()):
    # annotation이 없는 이미지 삭제
    if img_info["words"] == {}:
        del anno_temp[img_name]
        continue
    for obj, obj_info in img_info["words"].items():
        if len(img_info["words"][obj]["points"]) == 4:
            continue
        # 점이 홀수개인 텍스트박스 삭제
        elif len(img_info["words"][obj]["points"]) % 2 == 1:
            del anno_temp[img_name]["words"][obj]

            if anno_temp[img_name]["words"] == {}:
                del anno_temp[img_name]
                continue

        # polygon을 텍스트 시작과 끝에서 각각 2개의 점 추출
        else:
            if anno_temp[img_name]["words"][obj]["orientation"] == "Vertical":
                anno_temp[img_name]["words"][obj]["points"] = [
                    anno_temp[img_name]["words"][obj]["points"][0],
                    anno_temp[img_name]["words"][obj]["points"][1],
                    anno_temp[img_name]["words"][obj]["points"][
                        (len(anno_temp[img_name]["words"][obj]["points"]) // 2)
                    ],
                    anno_temp[img_name]["words"][obj]["points"][
                        (len(anno_temp[img_name]["words"][obj]["points"]) // 2 + 1)
                    ],
                ]
            else:
                [topleft, topright, bottomright, bottomleft] = [
                    anno_temp[img_name]["words"][obj]["points"][0],
                    anno_temp[img_name]["words"][obj]["points"][
                        (len(anno_temp[img_name]["words"][obj]["points"]) // 2 - 1)
                    ],
                    anno_temp[img_name]["words"][obj]["points"][
                        (len(anno_temp[img_name]["words"][obj]["points"]) // 2)
                    ],
                    anno_temp[img_name]["words"][obj]["points"][-1],
                ]

                left_side = (
                    (topleft[0] - bottomleft[0]) ** 2
                    + (topleft[1] - bottomleft[1]) ** 2
                ) ** 0.5
                right_side = (
                    (topright[0] - bottomright[0]) ** 2
                    + (topright[1] - bottomright[1]) ** 2
                ) ** 0.5

                idx = None
                max_dist = 0
                out_of_topside, out_of_bottomside = None, None

                # 윗변에서 사각형 바깥에 존재하는 점의 존재 여부 확인
                for i in range(len(anno_temp[img_name]["words"][obj]["points"]) // 2):
                    point = anno_temp[img_name]["words"][obj]["points"][i]
                    dist = cal_dist(
                        bottomright[0],
                        bottomright[1],
                        bottomleft[0],
                        bottomleft[1],
                        point[0],
                        point[1],
                    )
                    if dist > max(left_side, right_side) and dist > max_dist:
                        idx = i
                        max_dist = dist

                # 외부에 점이 존재하면 사각형 확장
                if idx:
                    out_of_topside = anno_temp[img_name]["words"][obj]["points"][idx]
                    new_topleft = get_crosspt(
                        out_of_topside[0],
                        out_of_topside[1],
                        topleft[0],
                        topleft[1],
                        topright[0],
                        topright[1],
                        topleft[0],
                        topleft[1],
                        bottomleft[0],
                        bottomleft[1],
                    )
                    new_topright = get_crosspt(
                        out_of_topside[0],
                        out_of_topside[1],
                        topleft[0],
                        topleft[1],
                        topright[0],
                        topright[1],
                        topright[0],
                        topright[1],
                        bottomright[0],
                        bottomright[1],
                    )

                # 아랫변에서 사각형 바깥에 존재하는 점의 존재 여부 확인
                idx = None
                max_dist = 0
                for i in range(
                    len(anno_temp[img_name]["words"][obj]["points"]) // 2,
                    len(anno_temp[img_name]["words"][obj]["points"]),
                ):
                    point = anno_temp[img_name]["words"][obj]["points"][i]
                    dist = cal_dist(
                        topleft[0],
                        topleft[1],
                        topright[0],
                        topright[1],
                        point[0],
                        point[1],
                    )
                    if dist > max(left_side, right_side) and dist > max_dist:
                        idx = i
                        max_dist = dist

                # 외부에 점이 존재하면 사각형 확장
                if idx:
                    out_of_bottomside = anno_temp[img_name]["words"][obj]["points"][idx]
                    new_bottomright = get_crosspt(
                        out_of_bottomside[0],
                        out_of_bottomside[1],
                        bottomright[0],
                        bottomright[1],
                        bottomleft[0],
                        bottomleft[1],
                        topright[0],
                        topright[1],
                        bottomright[0],
                        bottomright[1],
                    )
                    new_bottomleft = get_crosspt(
                        out_of_bottomside[0],
                        out_of_bottomside[1],
                        bottomright[0],
                        bottomright[1],
                        bottomleft[0],
                        bottomleft[1],
                        topleft[0],
                        topleft[1],
                        bottomleft[0],
                        bottomleft[1],
                    )

                if out_of_topside:
                    topleft = new_topleft
                    topright = new_topright
                if out_of_bottomside:
                    bottomright = new_bottomright
                    bottomleft = new_bottomleft

                anno_temp[img_name]["words"][obj]["points"] = [
                    topleft,
                    topright,
                    bottomright,
                    bottomleft,
                ]

anno = {"images": anno_temp}

ufo_dir = osp.join("../input/data/camper", "ufo")
with open(osp.join(ufo_dir, "train.json"), "w") as f:
    json.dump(anno, f, indent=4)

100%|██████████| 1510/1510 [00:00<00:00, 19938.61it/s]
